In [1]:
# # Install necessary libraries
# %pip install torch torchvision
# %pip install transformers
# %pip install pillow
# %pip install ftfy regex tqdm
# %pip install git+https://github.com/openai/CLIP.git
# %pip install pytorch-fid
# %pip install torch-fidelity
# %pip install pandas
# %pip install matplotlib
# %pip install numpy
# %pip install scikit-learn

# Import libraries
import os
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import zipfile
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import save_image
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from pytorch_fid import fid_score
from torch_fidelity import calculate_metrics
# Import libraries
import os
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import math


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


/home/hi_born/anaconda3/envs/mr_ads_gen/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [3]:
ad_dataset_path = 'AD_Dataset_with_ClassLabel_1.csv'
user_dataset_path = 'User_Dataset.csv'
images_folder_path = 'image_embeddings'


# Load CSV files
ad_dataset = pd.read_csv(ad_dataset_path)
user_dataset = pd.read_csv(user_dataset_path)

In [23]:
# Define paths
ad_dataset_path = 'AD_Dataset_with_ClassLabel_1.csv'
user_dataset_path = 'User_Dataset.csv'
image_embedding_folder = 'image_embeddings'
text_embedding_folder = 'text_embeddings'

# Load datasets
ad_dataset = pd.read_csv(ad_dataset_path)
user_dataset = pd.read_csv(user_dataset_path)

def find_text_embedding_path(ad_id):
    # Construct the expected file name based on the ad_id (adjust if necessary)
    embedding_filename = f"text_embedding_{int(ad_id)}.npy"  # Assuming this naming convention
    embedding_path = os.path.join(text_embedding_folder, embedding_filename)
    
    # Check if the file exists, if so return the path, else return None or a placeholder
    if os.path.exists(embedding_path):
        return embedding_path
    else:
        return None  # Or raise an error, or use a placeholder string


ad_dataset['text_embedding_path'] = ad_dataset['ad_id'].apply(find_text_embedding_path)

# Verify paths and dataset structure
print(ad_dataset[['ad_id', 'text_embedding_path']].head())
print(f"Image embedding folder contains: {len(os.listdir(image_embedding_folder))} files.")

# Function to load text embeddings from .npy files with error handling
def load_text_embedding(filepath):
    try:
        if os.path.exists(filepath):
            return np.load(filepath)
        else:
            print(f"File not found: {filepath}")
            return None
    except EOFError:
        print(f"EOFError: Unable to load {filepath} – File might be empty or corrupted.")
        return None
    except Exception as e:
        print(f"Unexpected error while loading {filepath}: {e}")
        return None

# Load text embeddings into the dataset with error handling
ad_dataset['text_embedding'] = ad_dataset['text_embedding_path'].apply(lambda path: load_text_embedding(path))

# Display a few loaded text embeddings for verification
print(f"Sample text embeddings:\n{ad_dataset['text_embedding'].head()}")


# Function to load image embeddings from .pt files
def load_image_embedding(filepath):
    if os.path.exists(filepath):
        return torch.load(filepath, map_location=device)
    else:
        print(f"Image embedding not found: {filepath}")
        return None

# Load text embeddings into the dataset
ad_dataset['text_embedding'] = ad_dataset['text_embedding_path'].apply(lambda path: load_text_embedding(path))

# Example: Display some loaded text embeddings
print(f"Loaded text embedding sample:\n{ad_dataset['text_embedding'].head()}")

# Function to construct the correct image embedding path using saved Drive structure
def find_image_embedding_file(image_id):
    embedding_filename = f"embedding_{image_id}.pt"
    embedding_path = os.path.join(image_embedding_folder, embedding_filename)

    if os.path.exists(embedding_path):
        return embedding_path
    else:
        raise FileNotFoundError(f"Image embedding not found for ID: {image_id}")

# Function to load a single image embedding from .pt file
def load_image_embedding(filepath):
    try:
        if os.path.exists(filepath):
            return torch.load(filepath, map_location=device)
        else:
            print(f"Image embedding not found: {filepath}")
            return None
    except Exception as e:
        print(f"Error loading embedding from {filepath}: {e}")
        return None

# Function to process image embeddings in batches
def load_image_embeddings(dataset, batch_size=16):
    num_batches = math.ceil(len(dataset) / batch_size)

    for batch_idx in range(num_batches):
        batch_data = dataset.iloc[batch_idx * batch_size:(batch_idx + 1) * batch_size]
        batch_tensors = []

        # Load each embedding in the batch
        for _, row in batch_data.iterrows():
            try:
                embedding_path = find_image_embedding_file(row['id'])  # Adjusted for correct path resolution
                img_embedding = load_image_embedding(embedding_path)
                if img_embedding is not None:
                    batch_tensors.append(img_embedding)
            except FileNotFoundError as e:
                print(e)
            except Exception as e:
                print(f"Unexpected error: {e}")

        if batch_tensors:
            # Stack tensors and move to GPU (if available)
            batch_tensors = torch.stack(batch_tensors).to(device)

            # Save the batch tensor to disk
            save_path = os.path.join(image_embedding_folder, f"loaded_batch_{batch_idx}.pt")
            torch.save(batch_tensors.cpu(), save_path)  # Save on CPU to free GPU memory
            print(f"Saved batch {batch_idx} embeddings to {save_path}")

            # Clear GPU memory after processing each batch
            del batch_tensors
            torch.cuda.empty_cache()

        # Display progress
        clear_output(wait=True)
        progress = (batch_idx + 1) / num_batches * 100
        print(f"Progress: {progress:.2f}% ({batch_idx + 1}/{num_batches} batches)")

# Start loading image embeddings
load_image_embeddings(ad_dataset, batch_size=16)

print("All embeddings loaded successfully.")


Progress: 100.00% (563/563 batches)
All embeddings loaded successfully.


In [24]:
ad_dataset.columns

Index(['ad_id', 'image_path', 'text', 'dimensions', 'Click-through(CTR)',
       'Cost per Click(CPC)', 'Return on Ad Spend(ROAS)',
       'Conversion(Value less than CTR)', 'Purchase_Conversion_rate',
       'ClassLabel_final', 'text_embedding_path', 'text_embedding'],
      dtype='object')

In [25]:
# Check the total number of embeddings
total_embeddings = len(ad_dataset['text_embedding'])
print(f"Total number of embeddings: {total_embeddings}")


Total number of embeddings: 9000


In [27]:
# Find the indices where text_embedding is not None and size is 768
correct_size_indices = ad_dataset[ad_dataset['text_embedding'].apply(lambda x: x is not None and len(x) == 768)].index
print(f"Indices with embedding size 768: {list(correct_size_indices)}")


Indices with embedding size 768: []


In [28]:
# Check the first few rows of the text_embedding column
print(ad_dataset['text_embedding'].head())


0    [[0.06809492, -0.12058817, 0.23107693, 0.02658...
1    [[0.10373393, -0.10898319, 0.5905778, 0.213023...
2    [[0.045375224, -0.4151973, 0.37503538, 0.05655...
3    [[-0.2609169, -0.20043637, 0.32284147, 0.13952...
4    [[0.07046847, -0.31986868, 0.058868058, 0.2812...
Name: text_embedding, dtype: object


In [29]:
# Flatten each embedding in the text_embedding column
ad_dataset['text_embedding'] = ad_dataset['text_embedding'].apply(lambda x: np.array(x).flatten())

# Now check the length of the flattened embeddings
embedding_lengths = ad_dataset['text_embedding'].apply(len)
print(embedding_lengths.head())  # To verify the sizes after flattening


0    768
1    768
2    768
3    768
4    768
Name: text_embedding, dtype: int64


In [30]:
# Check the length of each embedding and identify any that are not size 768
embedding_lengths = ad_dataset['text_embedding'].apply(lambda x: len(x) if x is not None else 0)
print(embedding_lengths.value_counts())


text_embedding
768    8999
1         1
Name: count, dtype: int64


In [31]:
# Replace embeddings of size 1 with zero vectors of size 768
ad_dataset['text_embedding'] = ad_dataset['text_embedding'].apply(lambda x: x if len(x) == 768 else np.zeros(768))

# Confirm that all embeddings are now of size 768
embedding_lengths = ad_dataset['text_embedding'].apply(len)
print(embedding_lengths.value_counts())  # This should only show 768 now


text_embedding
768    9000
Name: count, dtype: int64


In [32]:
# Initialize the scaler for numerical features
scaler = StandardScaler()

# Define batch size
BATCH_SIZE = 250  # Adjust as needed

# Extract numerical features
numerical_features = ['Conversion(Value less than CTR)',
                      'Cost per Click(CPC)',
                      'Return on Ad Spend(ROAS)']

# Fit the scaler on the entire numerical data
scaler.fit(ad_dataset[numerical_features])

def batch_generator(dataset, batch_size):
    """Generates batches of data to avoid memory overload."""
    num_samples = len(dataset)
    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        yield dataset.iloc[start_idx:end_idx]

def process_batch(batch):
    """Processes a single batch by normalizing numerical features and combining them."""
    # Extract and normalize numerical features, ensuring consistent DataFrame input
    numerical_data = scaler.transform(batch[numerical_features])

    # Extract categorical features
    categorical_features = batch.drop(
        columns=numerical_features + ['text', 'image_path', 'text_embedding', 'ad_id']
    )
    categorical_data = categorical_features.values

    # Extract text embeddings
    text_embeddings = np.vstack(batch['text_embedding'])

    # Combine all features for this batch
    return np.concatenate([numerical_data, text_embeddings, categorical_data], axis=1)

# Initialize a list to store the combined feature vectors
feature_vectors_list = []

# Process the dataset in batches
for batch in batch_generator(ad_dataset, BATCH_SIZE):
    batch_features = process_batch(batch)
    feature_vectors_list.append(batch_features)

# Concatenate all batch results into the final feature vector array
feature_vectors = np.vstack(feature_vectors_list)

print("Feature extraction completed successfully.")


Feature extraction completed successfully.


In [33]:
# Merge ad_dataset with user_dataset on 'ad_id'
merged_dataset = pd.merge(ad_dataset, user_dataset, on='ad_id', how='left')


In [34]:
# One-Hot Encode 'gender' and 'location'
merged_dataset = pd.get_dummies(merged_dataset, columns=['gender', 'location'])


In [36]:
# Clone the StyleGAN2 repository
# %git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
# %cd stylegan2-ada-pytorch
# %pip install click

# Import StyleGAN modules
import sys
sys.path.append('/content/stylegan2-ada-pytorch')
import dnnlib
import legacy


In [38]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import autocast, GradScaler
import dnnlib
import legacy  # Part of StyleGAN2-ADA repository

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Prepare feature vectors as PyTorch tensors (Assume `feature_vectors` is defined)
feature_tensors = torch.tensor(feature_vectors, dtype=torch.float32).to(device)
print(f"Feature Vectors Shape: {feature_tensors.shape}")

# Assume `batch_tensors` is loaded earlier into memory
print(f"Loaded Batch Tensors Shape: {batch_tensors.shape}")

# Create TensorDataset and DataLoader
BATCH_SIZE = 64  # Adjust based on your GPU capacity
dataset = TensorDataset(batch_tensors, feature_tensors)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

# Load pretrained StyleGAN2-ADA Generator and Discriminator
with dnnlib.util.open_url('https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl') as f:
    pretrained_network = legacy.load_network_pkl(f)
    G = pretrained_network['G_ema'].to(device)  # Generator
    D = pretrained_network['D'].to(device)      # Discriminator

# Instantiate the Conditioning Network
class ConditioningNetwork(nn.Module):
    def __init__(self, input_dim, conditioning_dim):
        super(ConditioningNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(True),
            nn.Linear(512, conditioning_dim)
        )

    def forward(self, x):
        return self.fc(x)

conditioning_dim = 512
input_dim = feature_vectors.shape[1]
conditioning_net = ConditioningNetwork(input_dim, conditioning_dim).to(device)

# Initialize optimizers
optimizer_G = torch.optim.Adam(conditioning_net.parameters(), lr=0.002, betas=(0.0, 0.99))
optimizer_D = torch.optim.Adam(D.parameters(), lr=0.002, betas=(0.0, 0.99))

# Initialize mixed precision scaler
scaler = GradScaler()

# Define checkpoint saving function
def save_checkpoint(epoch, G, D, optimizer_G, optimizer_D, scaler):
    os.makedirs('./checkpoints', exist_ok=True)
    torch.save({
        'epoch': epoch,
        'G_state_dict': G.state_dict(),
        'D_state_dict': D.state_dict(),
        'optimizer_G_state_dict': optimizer_G.state_dict(),
        'optimizer_D_state_dict': optimizer_D.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, f'./checkpoints/checkpoint_epoch_{epoch}.pth')

# Training Loop with Mixed Precision
num_epochs = 10  # Adjust as needed

for epoch in range(num_epochs):
    # Print statement to track the epoch progress
    print(f"Starting Epoch [{epoch+1}/{num_epochs}]...")

    for i, (real_images, features) in enumerate(data_loader):
        real_images, features = real_images.to(device), features.to(device)

        optimizer_D.zero_grad()
        optimizer_G.zero_grad()

        with autocast():
            # Generate conditioning inputs
            conditioning_inputs = conditioning_net(features)

            # Generate latent codes
            latent_codes = torch.randn(real_images.size(0), G.z_dim, device=device)

            # Generate fake images
            fake_images = G(latent_codes, c=conditioning_inputs)

            # Discriminator loss
            D_real = D(real_images, conditioning_inputs)
            D_fake = D(fake_images.detach(), conditioning_inputs)
            loss_D = -torch.mean(D_real) + torch.mean(D_fake)

            # Generator loss
            D_fake_G = D(fake_images, conditioning_inputs)
            loss_G = -torch.mean(D_fake_G)

        # Backpropagate discriminator loss
        scaler.scale(loss_D).backward()
        scaler.step(optimizer_D)

        # Backpropagate generator loss
        scaler.scale(loss_G).backward()
        scaler.step(optimizer_G)

        scaler.update()

        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(data_loader)}], "
                  f"Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

    # Save checkpoint at the end of each epoch
    save_checkpoint(epoch, G, D, optimizer_G, optimizer_D, scaler)

    # Print statement to indicate epoch completion
    print(f"Completed Epoch [{epoch+1}/{num_epochs}]")

print("Training completed successfully!")


Using device: cpu


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [39]:
# Inspect the contents of feature_vectors
print(f"Feature Vectors Data Type: {type(feature_vectors)}")
print(f"Feature Vectors Sample: {feature_vectors[:5]}")

Feature Vectors Data Type: <class 'numpy.ndarray'>
Feature Vectors Sample: [[-0.5755673978965404 -1.2161115807481238 0.9976056249796139 ... 2
  'comic_book' 'text_embeddings/text_embedding_1.npy']
 [2.465561307929546 -1.4631324780353672 -1.057857036482435 ... 3
  'web_site' 'text_embeddings/text_embedding_2.npy']
 [-0.7276238331878448 -0.6136689469523845 -0.35559445568784986 ... 1
  'web_site' 'text_embeddings/text_embedding_3.npy']
 [-0.7276238331878448 -0.5409317818424095 -1.075097259826736 ... 1
  'organ' 'text_embeddings/text_embedding_4.npy']
 [-0.7276238331878448 -0.38909401341167027 -0.5942288447759985 ... 0
  'web_site' 'text_embeddings/text_embedding_5.npy']]


In [40]:
import torch
import numpy as np

from torch.utils.data import DataLoader, TensorDataset

batch_size = 8  # Adjust based on your memory constraints
num_epochs = 10  # Number of epochs for training

def clean_feature_vectors(feature_vectors):
    """Convert all elements to floats, handling tuples, strings, and invalid entries."""
    cleaned_vectors = []
    for vector in feature_vectors:
        cleaned_vector = []
        for item in vector:
            if isinstance(item, str):
                try:
                    # Convert string that looks like a tuple, e.g., '(300, 250)'
                    item = eval(item)
                    if isinstance(item, tuple):
                        # Take the average of tuple values
                        item = sum(item) / len(item)
                except:
                    item = 0.0  # Default to 0.0 if conversion fails

            # Ensure item is a valid numeric type
            if isinstance(item, (int, float)):
                cleaned_vector.append(float(item))
            else:
                cleaned_vector.append(0.0)  # Default to 0.0 for invalid types

        cleaned_vectors.append(cleaned_vector)

    return np.array(cleaned_vectors, dtype=np.float32)

# Clean the feature vectors and ensure they are properly formatted
print(f"Original Feature Vectors Dimensions: {len(feature_vectors)}, {len(feature_vectors[0])}")
feature_vectors_cleaned = clean_feature_vectors(feature_vectors)

# Print cleaned feature vectors dimensions for verification
print(f"Cleaned Feature Vectors Dimensions: {feature_vectors_cleaned.shape}")

# Convert cleaned feature vectors to PyTorch tensors (CPU only)
feature_tensors = torch.tensor(feature_vectors_cleaned, dtype=torch.float32)

# Generate placeholder image tensors (adjust shape if needed)
image_tensors = torch.zeros(len(feature_tensors), 3, 256, 256, dtype=torch.float32)

# Print the dimensions of the image tensors
print(f"Image Tensors Dimensions: {image_tensors.shape}")

# Create DataLoader for efficient batch processing
dataset = TensorDataset(image_tensors, feature_tensors)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for i, (real_images, features) in enumerate(data_loader):
        # Ensure tensors are on CPU
        real_images = real_images.to('cpu', non_blocking=True)
        features = features.to('cpu', non_blocking=True)

        # Print batch dimensions (for debugging)
        print(f"Batch {i + 1} - Real Images: {real_images.shape}, Features: {features.shape}")

        # Generate conditioning inputs
        conditioning_inputs = conditioning_net(features)

        # Generate latent codes
        latent_codes = torch.randn(batch_size, G.z_dim)

        # Generate fake images with conditioning
        fake_images = G(latent_codes, c=conditioning_inputs)

        # Train Discriminator
        D_real = D(real_images, conditioning_inputs)
        D_fake = D(fake_images.detach(), conditioning_inputs)
        loss_D = -torch.mean(D_real) + torch.mean(D_fake)

        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        D_fake = D(fake_images, conditioning_inputs)
        loss_G = -torch.mean(D_fake)

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] completed. Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")


Original Feature Vectors Dimensions: 9000, 776
Cleaned Feature Vectors Dimensions: (9000, 776)
Image Tensors Dimensions: torch.Size([9000, 3, 256, 256])
Batch 1 - Real Images: torch.Size([8, 3, 256, 256]), Features: torch.Size([8, 776])


NameError: name 'conditioning_net' is not defined